<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning/blob/master/ML7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np

#학습 알고리즘
전제 : 신경망에는 적응 가능한 가중치와 편향이 있다. 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 **학습**이라고 한다.

**1단계 : 미니배치**

훈련데이터 중 일부를 무작위로 가져옴. 이 미니배치의 손실 함수 값을 줄이는게 목표

**2단계 : 기울기 산출** 

미니배치 손실 함수의 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함. 
기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시

**3단계 : 매개변수 갱신**

가중치 매개변수를 기울기 방\향으로 아주 조금 (학습률만큼) 갱신

**4단계 : 반복**

1~3단계의 반복


이를 확률적 경사 하강법이라고 함 ( Stochastic Gradient Descent) . SGD라고도 함

대부분의 딥러닝 프레임워크는 이 기능을 사용하고 있다

In [19]:
import sys, os
sys.path.append(os.pardir)
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std= 0.01):
    # input size : 입력층 뉴런 수 , hidden_size : 은닉층 뉴런 수 , output_size : 출력 층 뉴런 수 
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']
    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    y = softmax(a2)
    return y
    # y : 최종 출력층에서 활성화함수까지 돌린 결과값. 
    # x : 입력 데이터
    # t : 정답 레이블 
  def loss(self, x, t):
    y = self.predict(x)
    return  cross_entropy_error(y,t) # loss 함수로  교차엔트로피오차 를 사용한 모습.
    # 손실함수의 값을 리턴합니다
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y,axis=1) # 100개의 데이터가 한번의 dummy로 들어갔다면 100*1 shape의 행렬 추출
    t = np.argmax(t,axis=1) # 마찬가지
    accuracy = np.sum(y==t) / float(x.shape[0]) # 정확도 추출. numpy 배열 특성상 산술연산자 수행시 boolean 배열 추출. true이면 1 의 원리로 전체 accuracy 계산가능
    return accuracy
  def numerical_gradient(self, x, t): #여기서부터. 사실 여기서 numercial_gradient 함수에 이상이 있는것 같다
    # 사실 우연히 이름이 같은 것 뿐
    loss_W = lambda W : self.loss(x,t) #---> def loss_W(W) : return self.loss(x,t) 와 같은 말임 .
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 편미분 함수의 인자로 loss함수와 w 가중치가 들어가는 모습
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads
    # (loss함수에 대한) 가중치 매개변수의 기울기를 리턴.
    # params : 신경망의 매개변수를 보관하는 딕셔너리 변수 ----> 각 층의 가중치인 w, b의 값
    # grads : 기울기를 보관하는 딕셔너리 변수 . ---> loss함수에 대한 가중치의 기울기
  def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads
        # gradient 함수를 미리 끌고 왔습니다. 모든 문제가 해결되었습니다

In [20]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


np.random.rand(m,n) : 0~1의 균일분포 표준정규분포 난수로 matrix array(m,n) 생성

np.random.randn(m,n) : 평균 0, 표준편차 1의 가우시안 표준정규분포 난수로 matrix array(m,n(생성)

np.random.radint : __사이 랜덤숫자 1개 뽑아냄

In [21]:
x = np.random.rand(100,784) #더미 입력 데이터 100장 . 각 데이터는 28*28 크기라서 784행.
y = net.predict(x) # 예측 처리의 예시

In [22]:
x = np.random.rand(100,784)
t = np.random.rand(100,10)
#수정완료
grads = net.gradient(x,t)# 기울기 계산. 정확히는 loss함수에 대한 매개변수(가중치)의 기울기. 
#무슨 이유인지는 모르겠지만 윗줄에서 계속 프로그램이 멈춤. 아마 class 내부 numerical_gradient 작성에 근본 오류가 있는듯, 안끝남
#원본 코드를 가져왔는데도 실행이 안됬음.. 종결
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)
#아무튼 numerical_gradient()매서드를 이용하면 grads 변수에 [(loss함수에 대한 가중치의)기울기 정보]가 기록된다.
# 다음 장에서 gradient(self, x ,t )를 배우는데, 이게 희망인듯

(784, 100)
(100,)
(100, 10)
(10,)


#__Init__ 함수에서

input_size : 입력층의 뉴런 수 . 크기 하나가 가 28*28 이니까 784임. 

기본적으로 데이터가 하나일때를 기준으로 한다는것에 주목

hidden_size : 랜덤 (현 상황에서)

output_size : 0~9까지의 숫자 결과 인식이기 떄문에 10개



**++ 가중치 매개변수의 초기화.**

가중치 매개변수의 초기값을 어떻게 설정하느냐에 신경망 학습의 성공이 좌우된다

일단은

가중치 매개변수는 정규분포를 따르는 난수

편향은 0으로 초기화 한다 고 알고 있쟈

#트릭
grads = net.numerical_gradient(x,t) 

다시 말해 numerical_gradient(self,x,t)를 실행했을떄 실행이 안되는건 아니었다. 엄청 오래 걸려서 계속 로딩중이었던 것 뿐.

그래서 추후에 배우는게 gradient(self, x, t) 이다. 

이는 수치 미분 방식이 아니라 오차역전파법이다. 

같은 기울기를 구하지만 속도가 훨씬 빠르다.

그만큼 신경망 학습이 오래 걸리는 것이고, 

**그래서 numerical_gradient 대신 gradient 를 쓴다라고 깨우치면 될듯**

#미니배치 함수 구현

In [27]:
import numpy as np
from mnist import load_mnist
#from two_layer_net import TwoLayerNet
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)
train_loss_list = []

iters_num = 10000 #반복횟수
train_size = x_train.shape[0] #실제 배치 데이터의 갯수
batch_size = 100 # 미니 배치의 크기. 추후에 ,train_size를 batch_size로 바꿀듯.
learning_rate = 0.1 #학습률
network = TwoLayerNet(784, 50,10)

for i in range(iters_num):
  # 미니배치의 획득 과정입니다. x, t 로 들어갈 데이터의 갯수(batch)를 미니배치 시킨 과정입니다
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  #기울기 계산
  #grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch) 
  #이게 계속 말했던 성능 개선판입니다.

  #매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  #학습 경과 기록. 최종 loss를 구함. 
  #바뀐 weight 값이 여기서 쓰임
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)
#실행시키는데 38초 걸림

#numerical_gradient 혹은 gradient 함수안 인자에 도대체 뭐가 들어가는가?

TwoLayerNet 클래스에서 보면 알겠지만 numerical_gradient 함수 안에 numerical_gradient 함수가 또 있는 것을 알 수 있다. 

TwoLayerNet.numerical_gradient 매써드에서는 인자에 x, t 즉, 
입력 데이터와 정답 레이블이 들어감. 

그냥 numerical_gradient 매써드에서는 인자에 내가 알던대로 loss 함수와 가중치 매개변수들이 들어감 

#쉽게말해
전체 흐름에서 보면 만들어놓은 클래스의 인자에 최종적으로 알고 싶은 값, 

즉 입력 데이터와 결과 레이블을 넣어주는게 맞다


그래서 이 값들이 클래스 내부 "큰 " gradient 함수에 들어가주면 

이 함수내부에서 "작은" gradient 함수가 실행되고 이 작은 함수의 인자에 내가 알던대로 

loss 함수와 가중치 매개변수가 들어가는 것 으로 이해하면 될듯


#흐름


1.미니배치 크기를 100으로 하였다. 6만개의 데이터에서 임의로 100개의 데이터를 추려낸다

2. 100개의 미니배치를 대상으로 확률적 경사 하강법을 수행해 **매개변수**를 갱신함. 경사법에 의한 갱신 횟수를 10,000번으로 설정.

3. 각각 갱신될때마다(10,000번) 훈련 데이터에 대한 손실 함수를 계산하고 그 값을 배열에 추가함

#만약
학습 횟수가 늘어감에 따라 손실함수의 값이 줄어든다면

신경망의 가중치 매개변수가 서서히 데이터에 적응하고 있다.

#즉

신경망이 올바르게 학습되고 있다!

하지만 신경망 학습의 원래 목표인 범용성을 위해서는 훈련 데이터 외의 데이터를 올바르게 인식하는지 확인하는 과정이 필요

즉 오버피팅이 일어나지 않는지를 확인해야함

**epoch : 1 에폭은 학습에서 훈련 데이터를 모두 소진했을 때의 횟수 **

훈련 데이터 10,000개를 100개의 미니배치로 학습할 경우 확률적 경사 하강법을 100회 반복하면 모든 훈련 데이터를 소진한게 된다. 

**미니배치의 크기가 100이다  == 미니배치의 개수(데이터의 개수)가 100이다**


이경우 100회가 1 에폭임

--> 10,000/100  = 100. 간단히 전체 데이터 개수 / batch 크기

#그냥 간단히
epoch는 똑같은 학습과정을 몇번 반복하냐임
epoch = 100이면 똑같은 과정을 100번 반복하여 학습하라는 거임

In [30]:
import numpy as np
from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)


train_loss_list = []
train_acc_list = [] # !!
test_acc_list = [] # !!

iters_num = 10000 
train_size = x_train.shape[0] 
batch_size = 100 
learning_rate = 0.1 
network = TwoLayerNet(784, 50,10)

iter_per_epoch = max(train_size/batch_size , 1) # !!
#iter_per_epoch # 여기선 600임. 
# 그러니까 한 에포치당 몇번 반복하냐 --> 10,000개 데이터면 100크기의 미니배치로 한꺼번에 함수에 넣어버리고, 이 과정을 100번 반복하면 모든 데이터가 소진. 
# 100크기의 미니배치를 100번 반복해야지 모든 데이터에 대한 학습 한바퀴가 끝나지. 
# 여기서는 데이터가 6만개, 미니배치의 크기를 100으로 했으니 에포치당 반복횟수는 600. 

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  
  #grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch) 

  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0: # 한 애폭이 다 끝났을떄. 다시말해 1 에폭당 정확도 계산 식. 이 떄만 정확도를 구하겠다. 모든 i(반복횟수)마다 다하면 오래걸리고 그럴 필요도 x
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc :  " + str(train_acc) + ", " + str(test_acc))
# 실행은 됌. 다소 느릴뿐!! gradient로 그래도 그나마 빨라진거 확인가능

train acc, test acc :  0.09736666666666667, 0.0982
train acc, test acc :  0.7841166666666667, 0.7885
train acc, test acc :  0.8743833333333333, 0.8807


KeyboardInterrupt: ignored

#iter_per_epoch # 여기선 600임. 

그러니까 한 에포치당 몇번 반복하냐 

--> 10,000개 데이터면 100크기의 미니배치로 한꺼번에 함수에 넣어버리고, 이 과정을 100번 반복하면 모든 데이터가 소진. 

100크기의 미니배치를 100번 반복해야지 모든 데이터에 대한 학습 한바퀴가 끝나지. 

여기서는 데이터가 6만개, 미니배치의 크기를 100으로 했으니 에포치당 반복횟수는 600. 

#위 코드에서
i = 10,000이었고 iter_per_epoch는 600이었다.


i가 600의 배수, 즉 에폭에 해당하는 값일 경우에만 정확도를 계산했다

i는 반복문에 의해 1~10,000까지 차례로 증가한다. 그러다가 i가 애폭에 해당하는 값( 600의 배수) 가 될때만 정확도를 계산



i가 커지면서 학습이 진행

epoch가 커지면서 학습이 진행

**크게 봤을때는 i가 커지나 epoch가 커지나. 학습을 진행시키는 정도의 parameter인것은 분명하다**

만약 훈련데이터와 시험 데이터에 대한 정확도가 학습이 진행되는 정도에 따라
1에 수렴하고 있다면 , 오버피팅이 일어나지 않은 것이다

만약 시험 데이터의 정확도가 어느순간 떨어지고 있다면 오버피팅이 발생한 것이다

참고로 훈련데이터의 정확도는 무조건 계속 높아진다

**참고로 훈련 데이터로 학습한 모델의 범용 능력을 시험 데이터로 평가한다**

오버피팅 : 지나치게 훈련 데이터에 적응하여 시험 데이터에 대한 적절한 판단을 못하는 현상